In [1]:
import json

#ingresar las claves / scretas como cadenas en los siguientes campos
credentials = {}
credentials["CONSUMER_KEY"]= "yq6qDJh9rKMNZ9Y4hpMXbm1hX"
credentials["CONSUMER_SECRET"]= "gLwCTyUqFLBXwtqozU9th0TjlWq10sj2IrgTQLAn3BbGFjsLKk"
credentials["ACCESS_TOKEN"]= "1310810628899196929-26tj7I3ZhhEsfZeqWrXBQMwzZU47o9"
credentials["ACCESS_SECRET"]= "7H6UV3B62HcCzgMT72Wnv0CI7m0eK647vFltE22IeKYpp"

# guardar el objeto de credenciales en el archivo
with open("twitter_credentials.json", "w") as file:
  json.dump(credentials, file)

In [2]:
# importar la clase Twython
!pip install twython
from twython import Twython
import json

# cargar credenciales del archivo json
with open("twitter_credentials.json", "r") as file:
  creds = json.load(file)

# instanciar un objeto
python_tweets = Twython(creds["CONSUMER_KEY"], creds["CONSUMER_SECRET"])

# crea tu consulta utilizando la palabra "Microsoft"
query = {"q": "covid",
         "result_type": "popular",
         "count": 10,
         "lang": "en",
         }

In [3]:
import pandas as pd

#buscar tweets
dict_ = {"user": [], "date": [], "text": [], "favorite_count": []}
for status in python_tweets.search(**query)["statuses"]:
  dict_["user"].append(status["user"]["screen_name"])
  dict_["date"].append(status["created_at"])
  dict_["text"].append(status["text"])
  dict_["favorite_count"].append(status["favorite_count"])

# estructura de datos en un DataFrame de pandas para una manipulación más fácil
df = pd.DataFrame(dict_)
df.sort_values(by="favorite_count", inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count
0,realDonaldTrump,Fri Oct 30 12:22:37 +0000 2020,The @nytimes now admits that patients ill with...,94490
2,BigHitEnt,Fri Oct 30 13:43:52 +0000 2020,"[기사] #TXT Talk ""Blue Hour,"" Staying Connected ...",94303
4,kirstiealley,Fri Oct 30 11:18:40 +0000 2020,I now Know why my personal friends who walk ar...,52732
1,benshapiro,Thu Oct 29 11:59:02 +0000 2020,Covid is more dangerous than the flu (but not ...,44787
9,JoeBiden,Thu Oct 29 22:31:00 +0000 2020,You have my word: if I have the honor of servi...,36508


In [4]:
from twython import TwythonStreamer
import csv

# filtra datos no deseados
def process_tweet(tweet):
  d = {}
  d["hashtags"] = [hashtag["text"] for hashtag in tweet["entities"]["hashtags"]]
  d["text"] = tweet["text"]
  d["user"] = tweet["user"]["screen_name"]
  d["user_loc"] = tweet["user"]["location"]
  return d

# crear una clase qu herede TwythonStreame
class MyStreamer(TwythonStreamer):

#datos recividos
  def on_success(self, data):
      
    #solo recopila tweets en español
    if data["lang"] == "en":
      tweet_data = process_tweet(data)
      self.save_to_csv(tweet_data)

    
#problema con la API
  def on_error(self, status_code, data):
    print(status_code, data)
    self.disconnect()

  # GUARDAR CADA TWEETS EN UN ARCHIVO CSV
  def save_to_csv(self, tweet):
    with open(r"saved_tweets.csv", "a",encoding="utf-8") as file:
      writer = csv.writer(file)
      writer.writerow(list(tweet.values())) 

In [5]:
# IMPORTANTE: SOLO DEJAR CORRER POR UNOS MINUTOS Y LUEGO INTERRUMPIR LA EJECUCIÓN Y PASAR
# AL SIGUIENTE CÓDIGO A SEGUIR CORRIENDO, (saldrá un mensaje de error no le preste atención) (estos tweets se están almacenando en un csv que se puede descargar)

# instanciar desde nuestra clase de transmisión
stream = MyStreamer(creds["CONSUMER_KEY"], creds["CONSUMER_SECRET"],
                    creds["ACCESS_TOKEN"], creds["ACCESS_SECRET"])
# iniciar la transmisión
stream.statuses.filter(track="covid")

KeyError: ignored

In [6]:
import pandas as pd
tweets = pd.read_csv("saved_tweets.csv")
tweets.head()

,[],"@realDonaldTrump Over 3,600 new cases of COVID in MI today and another dozen deaths.",PoliticsN3rd,Here
0,[],Have none of his advisors corrected him on thi...,JCavallo972,"New Haven, CT"
1,[],RT @Argus911: One out of every 65 people in So...,JmeMiller1974,NaN
2,[],RT @gtconway3d: This is truly one of the sicke...,muriel_ferris,NaN
3,[],RT @GailSimone: Good lord. https://t.co/TyR2BC...,oodja,"Milford, CT"
4,[],And he just talked about Bieden hiding in a ba...,B_NERD,"Atlanta, GA"


In [7]:
# se obseva los 6 primeros tweets recolectados

import pandas as pd
tweets = pd.read_csv("saved_tweets.csv")
tweets.head()

,[],"@realDonaldTrump Over 3,600 new cases of COVID in MI today and another dozen deaths.",PoliticsN3rd,Here
0,[],Have none of his advisors corrected him on thi...,JCavallo972,"New Haven, CT"
1,[],RT @Argus911: One out of every 65 people in So...,JmeMiller1974,NaN
2,[],RT @gtconway3d: This is truly one of the sicke...,muriel_ferris,NaN
3,[],RT @GailSimone: Good lord. https://t.co/TyR2BC...,oodja,"Milford, CT"
4,[],And he just talked about Bieden hiding in a ba...,B_NERD,"Atlanta, GA"


In [8]:
# mostramos todos los tweets recolectados con la palabra "Microsoft"

from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv")
tweets

,[],"@realDonaldTrump Over 3,600 new cases of COVID in MI today and another dozen deaths.",PoliticsN3rd,Here
0,[],Have none of his advisors corrected him on thi...,JCavallo972,"New Haven, CT"
1,[],RT @Argus911: One out of every 65 people in So...,JmeMiller1974,NaN
2,[],RT @gtconway3d: This is truly one of the sicke...,muriel_ferris,NaN
3,[],RT @GailSimone: Good lord. https://t.co/TyR2BC...,oodja,"Milford, CT"
4,[],And he just talked about Bieden hiding in a ba...,B_NERD,"Atlanta, GA"
...,...,...,...,...
816,[],RT @bigbear5822: This shows that @GOP is more ...,rmitchell1600,NaN
817,[],Trump Dead Ass Exposed The Covid 19 bein ruled...,jayjay__93,NaN
818,[],RT @Corvette_9: Hell no!!! https://t.co/2MOGZ5...,epuujee,Mongolia
819,[],"RT @StandUpAmerica: Joey Traywick, a COVID-19 ...",pamelaowsinek,NaN


In [9]:
# aqui mostramos solamente el texto 
tweets.iloc[:, [False, True, False, False]]

,"@realDonaldTrump Over 3,600 new cases of COVID in MI today and another dozen deaths."
0,Have none of his advisors corrected him on thi...
1,RT @Argus911: One out of every 65 people in So...
2,RT @gtconway3d: This is truly one of the sicke...
3,RT @GailSimone: Good lord. https://t.co/TyR2BC...
4,And he just talked about Bieden hiding in a ba...
...,...
816,RT @bigbear5822: This shows that @GOP is more ...
817,Trump Dead Ass Exposed The Covid 19 bein ruled...
818,RT @Corvette_9: Hell no!!! https://t.co/2MOGZ5...
819,"RT @StandUpAmerica: Joey Traywick, a COVID-19 ..."
